# Training

This notebook demonstrates the process of training an ensemble learning model using a provided CSV file. It showcases data preprocessing, model training, evaluation, and saving the trained model. The ensemble method (hard voting, soft voting, or stacking) can be selected based on the user's choice.


## Importing Necessary Libraries

First, we import all the necessary libraries and modules needed for this script. This includes libraries for handling warnings, data manipulation, machine learning, and the custom Ensemble module containing ensemble learning methods.

In [1]:
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, 
    recall_score, 
    precision_score, 
    f1_score
)
from src import Bayes, Utils
import math

/tmp/ipykernel_39857/2614633084.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Utility Functions

In [2]:
MODEL_FOLDER = 'model_bayes'

### Function to Read CSV File

The `read_csv_file` function reads the CSV file and returns a pandas DataFrame. If the file is not found, the script will exit with an error message.

In [3]:
def read_csv_file(filename: str) -> pd.DataFrame:
    try:
        data = pd.read_csv(filename, lineterminator='\n', usecols=range(2))
        print("CSV file read successfully!")
        return data
    except FileNotFoundError:
        print("ERROR: File not found")
        exit(1)

# Demonstrate reading a CSV file (use a sample or mock filename)
# dataset = read_csv_file('datasets/datasetall.csv')

dataset = Utils.read_csv_file('datasets/datasetall.csv')
dataset

CSV file read successfully!


,text,label
0,Binay: Patuloy ang kahirapan dahil sa maling p...,0
1,SA GOBYERNONG TAPAT WELCOME SA BAGUO ANG LAHAT...,0
2,wait so ur telling me Let Leni Lead mo pero NY...,1
3,[USERNAME]wish this is just a nightmare that ...,0
4,doc willie ong and isko sabunutan po,0
...,...,...
28456,"Bisaya, Probinsyano/a, mostly Bisaya = katulong",1
28457,Amnesia. In my whole life wala pa ako nakasala...,1
28458,Kontrabida na ilang beses na tinalo at obvious...,1
28459,Yung antagonist laging kailangang sobrang sama...,1


In [4]:
dataset['label'].value_counts(ascending=True)

label
0    14115
1    14346
Name: count, dtype: int64

### Function to Seed Random Number Generators

To ensure reproducibility, the `seed_random_number_generators` function seeds the random number generators for PyTorch and NumPy.

In [5]:
def seed_random_number_generators(seed=0):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    print("Random number generators seeded.")

# Seed the random number generators
# seed_random_number_generators()

Utils.seed_random_number_generators()

Random number generators seeded.


### Function for Train-Test Split

The `get_train_test_split` function splits the dataset into training and testing sets with an 80-20 split ratio and returns them.

In [6]:
TEST_SIZE = 0.2

def shuffle_data_frame(data_frame):
    text = list(data_frame['text'])
    label = list(data_frame['label'])

    assert(len(text) == len(label))

    indices = list(range(len(label)))

    # Make a random number generator that will shuffle list of indices
    # It is seeded to be reproducible
    random_number_generator = np.random.default_rng(seed=0)
    random_number_generator.shuffle(indices)

    shuffled_text = []
    shuffled_labels = []

    # Iterate through the list of indices and add the original data
    # from those shuffled indices
    for index in indices:
        shuffled_text.append(text[index])
        shuffled_labels.append(label[index])

    return pd.DataFrame({
        'text': shuffled_text,
        'label': shuffled_labels,
    })


def get_train_test_split(data_frame: pd.DataFrame, test_size: float):
    """
    Makes a stratified train test split.
    This aims to preserve the distribution between classes.
    """
    if not (1 > test_size > 0):
        print('ERROR: test_size must be between 0 and 1')
        return

    data_frame = shuffle_data_frame(data_frame)

    data_frame_length = len(data_frame)
    train_size = 1 - test_size

    nonhate_rows = data_frame[data_frame['label'] == 0] 
    nonhate_row_length = len(nonhate_rows)

    nonhate_row_train_size = math.ceil(nonhate_row_length * train_size)

    nonhate_row_train = nonhate_rows[0:nonhate_row_train_size]
    nonhate_row_test = nonhate_rows[nonhate_row_train_size:nonhate_row_length]

    assert(len(nonhate_row_train) + len(nonhate_row_test) == nonhate_row_length)

    hate_rows = data_frame[data_frame['label'] == 1] 
    hate_row_length = len(hate_rows)

    hate_row_train_size = math.ceil(hate_row_length * train_size)

    hate_row_train = hate_rows[0:hate_row_train_size]
    hate_row_test = hate_rows[hate_row_train_size:hate_row_length]

    assert(len(hate_row_train) + len(hate_row_test) == hate_row_length)

    combined_train = pd.concat([nonhate_row_train, hate_row_train])
    combined_test = pd.concat([nonhate_row_test, hate_row_test])

    shuffled_train = shuffle_data_frame(combined_train)
    shuffled_test = shuffle_data_frame(combined_test)

    return (
        shuffled_train['text'],
        shuffled_test['text'],
        shuffled_train['label'],
        shuffled_test['label'],
    )

X_train, X_test, y_train, y_test = Utils.get_train_test_split(dataset, TEST_SIZE)

## Train Data

In [7]:
pd.DataFrame(X_train)

,text
0,[USERNAME] Palangga ka man sang mga taga Baco...
1,Who dafuq is Jose Montemayor Jr.???
2,Di na nakakatuwa yung mukha ni Mar Roxas sa TV...
3,national elections. | via[USERNAME]
4,"Binay will be staring in a movie called ""The D..."
...,...
22764,"""Kala ko wala andito pala si Marcos.""*pertaini..."
22765,sie ~ [USERNAME]Marcos Magnanakaw Marcos Dikta...
22766,If Mar is BatMarBinay is Bane-ay.
22767,to my moots im sorry in not sorry for flooding...


In [8]:
y_train_dataframe = pd.DataFrame(y_train, columns=['label'])
y_train_dataframe

,label
0,0
1,0
2,1
3,0
4,1
...,...
22764,0
22765,1
22766,1
22767,1


In [9]:
y_train_dataframe.value_counts(ascending=True)

label
0        11292
1        11477
Name: count, dtype: int64

## Test Data

In [10]:
pd.DataFrame(X_test)

,text
0,Bakit trending ang Only Binay?
1,Mare @ Cebu [USERNAME][USERNAME] Marcos Never ...
2,Kahit anong gawin ko bakit di ko ma appreciate...
3,Oras na para tayo'y bumoto ng taong mag tataas...
4,VP[USERNAME]is currently in Zamboanga Sibugay ...
...,...
5687,[USERNAME] Laban LeniAngat Buhay LahatLeni Kiko
5688,Nagconcede ka man Maimarwala ka prinnagdala ka...
5689,Did You Know that former Philippine secretary ...
5690,Bakit nakakairita commercial ni Mar Roxas?


In [11]:
VAL_SPLIT = 0.5

X_val, X_test, y_val, y_test = Utils.get_train_test_split(
  pd.DataFrame({
    'text': X_test,
    'label': y_test,
  }), 
  VAL_SPLIT,
)

In [12]:
y_test_dataframe = pd.DataFrame(y_test, columns=['label'])
y_test_dataframe

,label
0,0
1,0
2,0
3,0
4,0
...,...
2840,1
2841,0
2842,1
2843,1


In [13]:
y_test_dataframe.value_counts(ascending=True)

label
0        1411
1        1434
Name: count, dtype: int64

## Grid Search

### Bernoulli Naive Bayes

In [14]:
from sklearn.base import clone

alpha = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

total_accuracy = []
total_recall = []
total_precision = []
total_f1 = []
total_test_accuracy = []
total_test_recall = []
total_test_precision = []
total_test_f1 = []

for i in range(len(alpha)):
  train_bayes = clone(Bayes.BayesPipeline)

  train_bayes.set_params(
    bayes__alpha=alpha[i],
  )

  train_bayes.fit(X_train, y_train)

  accuracy, recall, precision, f1 = Utils.get_prediction_results(
    X_val,
    y_val,
    train_bayes,
  )

  test_accuracy, test_recall, test_precision, test_f1 = Utils.get_prediction_results(
    X_test,
    y_test,
    train_bayes,
  )

  total_accuracy.append(accuracy)
  total_recall.append(recall)
  total_precision.append(precision)
  total_f1.append(f1)
  total_test_accuracy.append(test_accuracy)
  total_test_recall.append(test_recall)
  total_test_precision.append(test_precision)
  total_test_f1.append(test_f1)

after count vocab
10168
  (0, 0)	4
  (0, 1)	2
  (0, 2)	2
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	2
  (0, 10)	5
  (0, 11)	2
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (0, 15)	1
  (0, 16)	1
  (0, 17)	1
  (0, 18)	1
  (0, 19)	1
  (0, 20)	1
  (1, 21)	1
  (1, 22)	1
  (1, 23)	1
  (1, 24)	1
  :	:
  (22767, 454)	1
  (22767, 479)	1
  (22767, 527)	1
  (22767, 563)	1
  (22767, 739)	1
  (22767, 740)	1
  (22767, 835)	1
  (22767, 847)	1
  (22767, 892)	1
  (22767, 911)	1
  (22767, 1114)	2
  (22767, 1151)	1
  (22767, 1597)	1
  (22767, 3395)	1
  (22767, 4604)	1
  (22767, 5662)	1
  (22767, 6533)	1
  (22767, 8026)	1
  (22767, 9913)	1
  (22767, 19536)	1
  (22767, 39662)	1
  (22768, 42)	1
  (22768, 99)	1
  (22768, 502)	1
  (22768, 5897)	1
before sort features
  (0, 0)	4
  (0, 1)	2
  (0, 2)	2
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	2
  (0, 10)	5
  (0, 11)	2
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (0, 15)	1
  (0, 16)	1
  (0, 17)	1
  (0, 18)	1
  (0

/home/nate/miniconda3/lib/python3.9/site-packages/sklearn/naive_bayes.py:624: FutureWarning: The default value for `force_alpha` will change to `True` in 1.4. To suppress this warning, manually set the value of `force_alpha`.
  warnings.warn(
/home/nate/miniconda3/lib/python3.9/site-packages/sklearn/naive_bayes.py:633: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(


after count vocab
34293
  (0, 15225)	1
  (0, 18050)	1
  (0, 18185)	1
  (0, 18367)	1
  (0, 25774)	1
  (0, 33981)	1
  (1, 7378)	1
  (1, 18050)	1
  (1, 18185)	1
  (1, 18367)	1
  (1, 25262)	1
  (1, 26573)	1
  (1, 32138)	1
  (2, 7296)	1
  (2, 18185)	1
  (2, 33299)	1
  (3, 1712)	1
  (3, 3428)	1
  (3, 3448)	1
  (3, 3457)	1
  (3, 5948)	1
  (3, 15868)	1
  (3, 36420)	1
  (3, 36552)	1
  (3, 37801)	2
  :	:
  (2841, 28617)	1
  (2841, 32107)	2
  (2841, 37562)	1
  (2841, 37801)	1
  (2842, 4454)	1
  (2842, 21092)	1
  (2843, 15868)	1
  (2843, 22793)	1
  (2843, 31500)	1
  (2844, 3873)	1
  (2844, 7827)	1
  (2844, 29527)	1
  (2844, 36552)	1
  (2845, 14665)	1
  (2845, 19612)	1
  (2845, 21092)	1
  (2845, 36420)	1
  (2845, 39589)	1
  (2846, 1712)	1
  (2846, 12067)	1
  (2846, 16774)	1
  (2846, 18050)	2
  (2846, 18185)	2
  (2846, 18367)	2
  (2846, 18791)	1
10957
Accuracy: 0.7801194239550404
Recall: 0.8264808362369338
Precision: 0.7587971849008317
F1-score: 0.791194129419613
after count vocab
34293
  (0, 1773)	

after count vocab
10168
  (0, 0)	4
  (0, 1)	2
  (0, 2)	2
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	2
  (0, 10)	5
  (0, 11)	2
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (0, 15)	1
  (0, 16)	1
  (0, 17)	1
  (0, 18)	1
  (0, 19)	1
  (0, 20)	1
  (1, 21)	1
  (1, 22)	1
  (1, 23)	1
  (1, 24)	1
  :	:
  (22767, 454)	1
  (22767, 479)	1
  (22767, 527)	1
  (22767, 563)	1
  (22767, 739)	1
  (22767, 740)	1
  (22767, 835)	1
  (22767, 847)	1
  (22767, 892)	1
  (22767, 911)	1
  (22767, 1114)	2
  (22767, 1151)	1
  (22767, 1597)	1
  (22767, 3395)	1
  (22767, 4604)	1
  (22767, 5662)	1
  (22767, 6533)	1
  (22767, 8026)	1
  (22767, 9913)	1
  (22767, 19536)	1
  (22767, 39662)	1
  (22768, 42)	1
  (22768, 99)	1
  (22768, 502)	1
  (22768, 5897)	1
before sort features
  (0, 0)	4
  (0, 1)	2
  (0, 2)	2
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	2
  (0, 10)	5
  (0, 11)	2
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (0, 15)	1
  (0, 16)	1
  (0, 17)	1
  (0, 18)	1
  (0

In [15]:
metrics_data_frame = pd.DataFrame(
  {
    'accuracy': total_accuracy,
    'recall': total_recall,
    'precision': total_precision,
    'f1': total_f1,
    'test_accuracy': total_test_accuracy,
    'test_recall': total_test_recall,
    'test_precision': total_test_precision,
    'test_f1': total_test_f1,
  },
  columns=[
    'accuracy', 
    'recall', 
    'precision', 
    'f1',
    'test_accuracy', 
    'test_recall', 
    'test_precision', 
    'test_f1',
  ],
)
metrics_data_frame.to_csv(f'{MODEL_FOLDER}/eval_metrics.csv')